In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Location of dataset on machine
personal_file_base = "C:/Users/laela/large_datasets/CSE 543 Dataset/"

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
# Create dataframe from CSV files

# Load the CSV file into a pandas DataFrame
shared_file_base = "MachineLearningCSV/MachineLearningCVE/"
all_df_paths = ["Monday-WorkingHours.pcap_ISCX.csv", "Tuesday-WorkingHours.pcap_ISCX.csv", "Wednesday-workingHours.pcap_ISCX.csv", "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv","Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv", "Friday-WorkingHours-Morning.pcap_ISCX.csv", "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv", "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"]
df_paths = ["Wednesday-workingHours.pcap_ISCX.csv", "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"]
df = pd.DataFrame()
for df_path in df_paths:
    day_df = pd.read_csv(df_path)
    df = pd.concat([df, day_df], ignore_index=True)
print("Dataframe is of shape", df.shape)

In [ ]:
# Dataframe preprocessing

# Removes unnecessary white space from column names
df.columns = df.columns.str.strip()

features_all_0 = ["Bwd PSH Flags", "Fwd URG Flags", "Bwd URG Flags", "CWE Flag Count", "Fwd Avg Bytes/Bulk", "Fwd Avg Packets/Bulk", "Fwd Avg Bulk Rate", "Bwd Avg Bytes/Bulk", "Bwd Avg Packets/Bulk", "Bwd Avg Bulk Rate"]
features_dervived = ['Flow Bytes/s', 'Flow Packets/s', 'Fwd Packets/s', 'Bwd Packets/s']
features_categorical = ['Label']
#features_negative_values = ['Flow Duration', 'Flow IAT Mean', 'Flow IAT Max', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'min_seg_size_forward']
#features_large_negative_values = ['Flow IAT Min', 'Fwd IAT Min', 'Fwd Header Length', 'Fwd Header Length.1']

# Removes some features
df_feature_selection = df.drop(columns=features_all_0, axis=1)
df_feature_selection = df_feature_selection.drop(columns=features_dervived, axis=1)

# Remove samples with invalid features (about half of samples)
df_feature_selection = df_feature_selection.replace(range(-14, 0), np.nan).dropna()

# Converts categorical data into numerical data
feature_mappings = {}
for feature in features_categorical:
    label_encoder = LabelEncoder()
    df_feature_selection[feature] = label_encoder.fit_transform(df_feature_selection[feature])
    mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
    print(f"Mapping for feature '{feature}': {mapping}") # 'BENIGN': 440031+97718=537749, 'DDoS': 128027, 'DoS GoldenEye': 10293, 'DoS Hulk': 231973, 'DoS Slowhttptest': 5499, 'DoS slowloris': 5796, 'Heartbleed': 11
    for index, value in mapping.items():
        feature_mappings[value] = index

# Converts port numbers from categorical to numerical
def replace_port_numbers(x):
    return 1 if x == 80 else 0
df_feature_selection['Destination Port'] = df_feature_selection['Destination Port'].apply(replace_port_numbers)

X = df_feature_selection.drop(['Label'], axis=1)
y = df_feature_selection['Label']

# Compute the correlation of each feature with the target
correlations = abs(X.corrwith(pd.Series(y)))

# Display the correlations
pd.set_option('display.max_rows', None) # Display all rows
correlations = correlations.sort_values(ascending=False)

X = X.drop(correlations.index[26:], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)